In [1]:
import os
os.chdir("../")
os.getcwd()

'/home/izam/coding/GemInsights'

In [2]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class PromptingConfig:
    root_dir: Path
    model_name: str
    response_file_name: str
    candidates_file_name: str
    credentials: Path
    generation_config: dict
    project_name: str
    project_location: str
    prompt_file_path: Path

    images_file_path: Path
    trulens_db_name: str
    

In [3]:
from gemInsights.constants import *
from gemInsights.utils.common import read_yaml, create_directories, load_json

In [4]:
#udpate configuration manager
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        credentials_file_path = CREDENTIALS_FILE_PATH,
        params_file_path = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.credentials = credentials_file_path
        self.params = read_yaml(params_file_path)

        create_directories([self.config.artifacts_root])


    def get_promting_config(self) -> PromptingConfig:
        config = self.config.prompting
        
        create_directories([config.root_dir])

        prompting_config = PromptingConfig(
            root_dir=config.root_dir,
            model_name=config.model_name, 
            response_file_name=config.response_file_name,
            candidates_file_name=config.candidates_file_name,
            credentials=self.credentials,
            generation_config=dict(self.params.generation_config),
            project_name=config.project_name,
            project_location=config.project_location,
            prompt_file_path=config.prompt_file_path,
            images_file_path=config.images_file_path,
            trulens_db_name=config.trulens_db_name,
        )

        return prompting_config

In [5]:
from google.cloud import aiplatform
from gemInsights import logger
import os
import base64
import vertexai
from vertexai.preview.generative_models import GenerativeModel, Part
from gemInsights.utils.common import load_json, save_json, load_bin, read_text
from trulens_eval import Feedback, Tru, LiteLLM, Huggingface, TruBasicApp

In [6]:
class Prompting:
    def __init__(self, config: PromptingConfig):
        self.config = config

    def _setup_env(self):
        tru = Tru(database_file=self.config.trulens_db_name)
        tru.reset_database()
        logger.info(f"initialized trulens with db - {self.config.trulens_db_name}")

        aiplatform.init(
            project = self.config.project_name,
            location= self.config.project_location,
        )
        logger.info(f"Google cloud project name - {self.config.project_name}")
        
        os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = self.config.credentials
        logger.info("loaded the google cloud credentials")

        self.model = GenerativeModel(self.config.model_name)
        logger.info(f"using the model - {self.config.model_name}")

        hugs = Huggingface()
        self.f_sentiment = Feedback(hugs.positive_sentiment).on_output()
        logger.info(f"initialized huggingface based sentiment feedback")

    def _initiate_data(self):
        print(f"{self.config.prompt_file_path}")
        self.prompt = read_text(Path(self.config.prompt_file_path))
        self.images = load_bin(Path(self.config.images_file_path))


    def get_response(self):
        self._setup_env()
        self._initiate_data()
        
        model = self.model
        images = self.images
        logger.info(type(self.images))
        configuration = self.config.generation_config

        def _llm_standalone(prompt):
            return model.generate_content(
                [prompt]+images , generation_config=configuration).text
        
        tru_app_recorder = TruBasicApp(_llm_standalone, app_id="Sentiment bot", feedbacks=[self.f_sentiment])
        logger.info("created the basic recorder app")

        logger.info(f"generating response with config - {self.config.generation_config}")
        with tru_app_recorder as records:
            response = tru_app_recorder.app("generate something")
        
        save_json(path=os.path.join(self.config.root_dir, self.config.response_file_name), data={"response": response})
        logger.info(response)

In [ ]:
try:
    config = ConfigurationManager()
    prompting_config = config.get_promting_config()
    prompting_config = Prompting(config=prompting_config)
    prompting_config.get_response()
except Exception as e:
    raise e

[2023-12-22 01:50:55,228: INFO: common: yaml file: config/config.yaml loaded successfully]
[2023-12-22 01:50:55,229: INFO: common: yaml file: params.yaml loaded successfully]
[2023-12-22 01:50:55,229: INFO: common: created directory at: artifacts]
[2023-12-22 01:50:55,230: INFO: common: created directory at: artifacts/prompting]
🦑 Tru initialized with db url sqlite:///gemInsights.sqlite .
🛑 Secret keys may be written to the database. See the `database_redact_keys` option of `Tru` to prevent this.
[2023-12-22 01:50:55,237: INFO: migration: Context impl SQLiteImpl.]
[2023-12-22 01:50:55,238: INFO: migration: Will assume non-transactional DDL.]
[2023-12-22 01:50:55,250: INFO: sqlalchemy_db: Deleted 2 rows.]
[2023-12-22 01:50:55,251: INFO: 948873657: initialized trulens with db - gemInsights.sqlite]
[2023-12-22 01:50:55,251: INFO: 948873657: Google cloud project name - ultra-heading-407815]
[2023-12-22 01:50:55,252: INFO: 948873657: loaded the google cloud credentials]
[2023-12-22 01:50:55

/tmp/ipykernel_97649/948873657.py:6: DeprecationWarning: `database_file` is deprecated, use `database_url` instead as in `database_url='sqlite:///filename'.
  tru = Tru(database_file=self.config.trulens_db_name)


[2023-12-22 01:51:14,410: INFO: migration: Context impl SQLiteImpl.]
[2023-12-22 01:51:14,411: INFO: migration: Will assume non-transactional DDL.]
[2023-12-22 01:51:14,416: INFO: sqlalchemy_db: ✅ added record record_hash_7950c362e1eeeee2f9c47d0620abc2c5]
[2023-12-22 01:51:14,426: INFO: migration: Context impl SQLiteImpl.]
[2023-12-22 01:51:14,426: INFO: migration: Will assume non-transactional DDL.]
[2023-12-22 01:51:14,479: INFO: common: json file saved at: artifacts/prompting/response.json]
[2023-12-22 01:51:14,481: INFO: 948873657:  The first graph shows a scatter plot of the churn rate vs monthly charges. It is difficult to see a clear pattern in this graph, although it does appear that there may be a positive correlation between the two variables. The second graph shows box plots of the churn rate vs monthly charges. This graph shows that the churn rate is higher for customers with higher monthly charges. The third graph shows a bar chart of the average monthly charges for custom

[2023-12-22 01:51:15,461: INFO: migration: Context impl SQLiteImpl.]
[2023-12-22 01:51:15,461: INFO: migration: Will assume non-transactional DDL.]
[2023-12-22 01:51:15,468: INFO: sqlalchemy_db: ✅ feedback result positive_sentiment DONE feedback_result_hash_b90451da18f803a8be99b1eb550f2851]
